In [32]:
%%writefile task1.c
#include <stdio.h>
#include <stdlib.h>  // For malloc() and free()
#include <mpi.h>     // For all MPI functions

#define MAX_ARRAY_SIZE 2048

int main(int argc, char** argv) {
    // Variable declarations
    int array_size = 10000;    // Default array size (64 elements)
    int *array = NULL;      // Pointer to our data array
    int comm_sz;            // Total number of MPI processes
    int my_rank;            // Current process ID (0 to comm_sz-1)
    double start_time, end_time, broadcast_time; // For timing measurements

    /* 1. MPI INITIALIZATION */
    MPI_Init(&argc, &argv);                     // Start MPI
    MPI_Comm_size(MPI_COMM_WORLD, &comm_sz);    // Get total processes
    MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);    // Get current process ID

    /* 2. HANDLE COMMAND LINE ARGUMENT (ARRAY SIZE) */
    if (argc > 1) {
        array_size = atoi(argv[1]);  // Get size from command line

        // Validate size is power of two using bitwise trick
        if ((array_size & (array_size - 1)) != 0) {
            if (my_rank == 0) {  // Only root prints error
                printf("Error: Array size must be power of two (e.g., 32, 64, 128)\n");
            }
            MPI_Finalize();
            return 1;
        }


        if (array_size > MAX_ARRAY_SIZE) {
            if (my_rank == 0) {
                printf("Error: Maximum array size is %d\n", MAX_ARRAY_SIZE);
            }
            MPI_Finalize();
            return 1;
        }
    }

    /* 3. ARRAY INITIALIZATION (ON ROOT PROCESS) */
    if (my_rank == 0) {
        // Allocate memory on root process
        array = (int*)malloc(array_size * sizeof(int));
        if (array == NULL) {
            printf("Root process: Memory allocation failed!\n");
            MPI_Abort(MPI_COMM_WORLD, 1);  // Emergency shutdown
        }

        // Fill array with values 0, 1, 2,... array_size-1
        for (int i = 0; i < array_size; i++) {
            array[i] = i;
        }
        printf("Root initialized array of size %d\n", array_size);
    }

    /* 4. MEMORY ALLOCATION ON OTHER PROCESSES */
    if (my_rank != 0) {
        array = (int*)malloc(array_size * sizeof(int));
        if (array == NULL) {
            printf("Process %d: Memory allocation failed!\n", my_rank);
            MPI_Abort(MPI_COMM_WORLD, 1);
        }
    }

    /* 5. SYNCHRONIZE BEFORE TIMING */
    MPI_Barrier(MPI_COMM_WORLD);  // All processes wait here until everyone arrives

    /* 6. TIME THE BROADCAST OPERATION */
    if (my_rank == 0) {
        start_time = MPI_Wtime();  // Start timer (root only)
    }

    /* 7. PERFORM THE BROADCAST */
    // MPI_Bcast arguments:
    // - array: Pointer to data
    // - array_size: Number of elements
    // - MPI_INT: Data type
    // - 0: Root process (sender)
    // - MPI_COMM_WORLD: All processes participate
    MPI_Bcast(array, array_size, MPI_INT, 0, MPI_COMM_WORLD);

    /* 8. CALCULATE AND PRINT BROADCAST TIME (ROOT ONLY) */
    if (my_rank == 0) {
        end_time = MPI_Wtime();
        broadcast_time = end_time - start_time;
        printf("Broadcast completed in %.6f seconds\n", broadcast_time);
    }

    /* 9. VERIFY DATA RECEPTION (ALL PROCESSES) */
    // Print first two and last two elements to verify correct transfer
    printf("Process %d received: [0]=%d, [1]=%d, ..., [%d]=%d, [%d]=%d\n",
           my_rank,
           array[0], array[1],
           array_size-2, array[array_size-2],
           array_size-1, array[array_size-1]);

    /* 10. CLEANUP AND EXIT */
    free(array);         // Free allocated memory
    MPI_Finalize();      // Shut down MPI
    return 0;
}


Overwriting task1.c


In [31]:
!mpicc -fopenmp task1.c -o task1
!./task1

Root initialized array of size 5000
Broadcast completed in 0.000002 seconds
Process 0 received: [0]=0, [1]=1, ..., [4998]=4998, [4999]=4999


In [18]:
%%writefile task2.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>   // For OpenMP functions
#include <time.h>  // For time() and rand()

#define ARRAY_SIZE 4096  // Using power-of-two size for better parallel performance

/**
 * Fills an array with random numbers between 0 and 9999
 * @param arr Pointer to the array
 * @param size Size of the array
 */
void fill_array(int* arr, int size) {
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 10000;  // Random numbers 0-9999
    }
}

/**
 * Copies elements from source array to destination array
 * @param src Source array
 * @param dest Destination array
 * @param size Size of arrays
 */
void copy_array(int* src, int* dest, int size) {
    for (int i = 0; i < size; i++) {
        dest[i] = src[i];  // Element-wise copy
    }
}

/**
 * Serial implementation of bubble sort
 * @param arr Array to sort
 * @param size Size of array
 */
void bubble_sort_serial(int* arr, int size) {
    // Outer loop: number of passes needed
    for (int i = 0; i < size - 1; i++) {
        // Inner loop: compare adjacent elements
        for (int j = 0; j < size - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                // Swap if out of order
                int tmp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = tmp;
            }
        }
    }
}

/**
 * Parallel bubble sort using Odd-Even Transposition algorithm
 * @param arr Array to sort
 * @param size Size of array
 */
void bubble_sort_parallel(int* arr, int size) {
    // Alternate between odd and even phases
    for (int phase = 0; phase < size; phase++) {
        if (phase % 2 == 0) {
            // Even phase: compare elements at even indices
            #pragma omp parallel for default(none) shared(arr, size)
            for (int i = 0; i < size - 1; i += 2) {
                if (arr[i] > arr[i + 1]) {
                    int tmp = arr[i];
                    arr[i] = arr[i + 1];
                    arr[i + 1] = tmp;
                }
            }
        } else {
            // Odd phase: compare elements at odd indices
            #pragma omp parallel for default(none) shared(arr, size)
            for (int i = 1; i < size - 1; i += 2) {
                if (arr[i] > arr[i + 1]) {
                    int tmp = arr[i];
                    arr[i] = arr[i + 1];
                    arr[i + 1] = tmp;
                }
            }
        }
    }
}

int main() {
    // Allocate memory for arrays
    int* original = (int*)malloc(sizeof(int) * ARRAY_SIZE);
    int* serial_arr = (int*)malloc(sizeof(int) * ARRAY_SIZE);
    int* parallel_arr = (int*)malloc(sizeof(int) * ARRAY_SIZE);

    // Initialize random number generator and fill original array
    srand(time(NULL));
    fill_array(original, ARRAY_SIZE);

    // Create copies for serial and parallel sorting
    copy_array(original, serial_arr, ARRAY_SIZE);
    copy_array(original, parallel_arr, ARRAY_SIZE);

    /* SERIAL SORTING */
    double start_serial = omp_get_wtime();  // Start timer
    bubble_sort_serial(serial_arr, ARRAY_SIZE);
    double end_serial = omp_get_wtime();    // End timer

    /* PARALLEL SORTING */
    double start_parallel = omp_get_wtime();  // Start timer
    bubble_sort_parallel(parallel_arr, ARRAY_SIZE);
    double end_parallel = omp_get_wtime();    // End timer

    // Print timing results
    printf("Serial Bubble Sort Time:   %f seconds\n", end_serial - start_serial);
    printf("Parallel Bubble Sort Time: %f seconds\n", end_parallel - start_parallel);

    /* RESULT VERIFICATION */
    int is_same = 1;  // Assume arrays are identical
    for (int i = 0; i < ARRAY_SIZE; i++) {
        if (serial_arr[i] != parallel_arr[i]) {
            is_same = 0;  // Found mismatch
            break;
        }
    }

    if (is_same) {
        printf("✔️ Both versions produced the same sorted array.\n");
    } else {
        printf("❌ Arrays differ. Parallel sort might be incorrect.\n");
    }

    // Clean up memory
    free(original);
    free(serial_arr);
    free(parallel_arr);

    return 0;
}

Writing task2.c


In [28]:
!gcc -fopenmp task2.c -o task2
!./task2

Serial Bubble Sort Time:   0.036906 seconds
Parallel Bubble Sort Time: 0.030490 seconds
✔️ Both versions produced the same sorted array.


In [20]:
%%writefile task3.cu



#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

// Macro for error checking CUDA API calls
#define CHECK(call) {\
    const cudaError_t error = call;\
    if (error != cudaSuccess) {\
        printf("CUDA Error: %s:%d, ", __FILE__, __LINE__);\
        printf("Code:%d, Reason: %s\n", error, cudaGetErrorString(error));\
        exit(1);\
    }\
}

/**
 * CUDA Kernel for Bitonic Sort Step
 *
 * @param dev_values Pointer to device array being sorted
 * @param j          Half the length of the current sorting sequence
 * @param k          Full length of the current bitonic sequence
 */
__global__ void bitonicSortStep(int *dev_values, int j, int k) {
    // Calculate global thread index
    unsigned int i = threadIdx.x + blockDim.x * blockIdx.x;
    // Calculate index of element to compare with
    unsigned int ixj = i ^ j;  // XOR operation for efficient index calculation

    // Only perform swap if the other element is in our array range
    if (ixj > i) {
        // Determine sort direction (ascending or descending)
        if ((i & k) == 0) {
            // Ascending order
            if (dev_values[i] > dev_values[ixj]) {
                // Swap elements
                int temp = dev_values[i];
                dev_values[i] = dev_values[ixj];
                dev_values[ixj] = temp;
            }
        }
        if ((i & k) != 0) {
            // Descending order
            if (dev_values[i] < dev_values[ixj]) {
                // Swap elements
                int temp = dev_values[i];
                dev_values[i] = dev_values[ixj];
                dev_values[ixj] = temp;
            }
        }
    }
}

/**
 * Verify if array is sorted in ascending order
 *
 * @param arr  Array to check
 * @param size Size of array
 * @return     True if sorted, False otherwise
 */
bool isSorted(int *arr, int size) {
    for (int i = 1; i < size; i++) {
        if (arr[i-1] > arr[i]) {
            return false;
        }
    }
    return true;
}

/**
 * Print portion of array contents (first and last 10 elements for large arrays)
 *
 * @param arr  Array to print
 * @param size Size of array
 */
void printArrayPortion(int *arr, int size) {
    if (size <= 20) {
        // Print entire array if small
        printf("Array contents: ");
        for (int i = 0; i < size; i++) {
            printf("%d ", arr[i]);
        }
    } else {
        // Print first 10 and last 10 elements for large arrays
        printf("First 10 elements: ");
        for (int i = 0; i < 10; i++) {
            printf("%d ", arr[i]);
        }
        printf("\nLast 10 elements: ");
        for (int i = size - 10; i < size; i++) {
            printf("%d ", arr[i]);
        }
    }
    printf("\n");
}

/**
 * Main function
 */
int main(int argc, char **argv) {
    // Default array size (must be power of two)
    int size = 2048;

    // Parse command line argument for array size if provided
    if (argc > 1) {
        size = atoi(argv[1]);
        // Validate that size is a power of two
        if ((size & (size - 1)) != 0) {
            printf("Error: Array size must be a power of two (e.g., 256, 512, 1024, 2048)\n");
            return 1;
        }
    }

    printf("Bitonic Sort - Array size: %d (must be power of two)\n", size);

    // Allocate host memory
    int *h_values = (int *)malloc(size * sizeof(int));
    if (h_values == NULL) {
        printf("Host memory allocation failed!\n");
        return 1;
    }

    // Initialize array with random numbers (0-999)
    srand(time(NULL));
    for (int i = 0; i < size; i++) {
        h_values[i] = rand() % 1000;
    }

    // Print initial array portion
    printf("\nInitial array:\n");
    printArrayPortion(h_values, size);

    // Allocate device memory
    int *d_values;
    CHECK(cudaMalloc((void **)&d_values, size * sizeof(int)));

    // Copy data from host to device
    CHECK(cudaMemcpy(d_values, h_values, size * sizeof(int), cudaMemcpyHostToDevice));

    // Create CUDA events for timing measurement
    cudaEvent_t start, stop;
    CHECK(cudaEventCreate(&start));
    CHECK(cudaEventCreate(&stop));

    // Record start time
    CHECK(cudaEventRecord(start, 0));

    // Configure kernel launch parameters
    int threads_per_block = 256;
    int blocks = (size + threads_per_block - 1) / threads_per_block;

    // Perform bitonic sort
    // Outer loop: build sequences of increasing size (2, 4, 8... up to 'size')
    for (int k = 2; k <= size; k <<= 1) {
        // Inner loop: merge sequences
        for (int j = k >> 1; j > 0; j >>= 1) {
            bitonicSortStep<<<blocks, threads_per_block>>>(d_values, j, k);
        }
    }

    // Check for kernel errors
    CHECK(cudaGetLastError());

    // Record stop time
    CHECK(cudaEventRecord(stop, 0));
    CHECK(cudaEventSynchronize(stop));

    // Calculate elapsed time
    float elapsedTime;
    CHECK(cudaEventElapsedTime(&elapsedTime, start, stop));

    // Copy sorted array back to host
    CHECK(cudaMemcpy(h_values, d_values, size * sizeof(int), cudaMemcpyDeviceToHost));

    // Verify sorting result
    if (isSorted(h_values, size)) {
        printf("\nSorting verification: SUCCESS\n");
    } else {
        printf("\nSorting verification: FAILED\n");
    }

    // Print sorted array portion
    printf("\nSorted array:\n");
    printArrayPortion(h_values, size);

    // Print performance results
    printf("\nPerformance metrics:\n");
    printf("- Array size: %d elements\n", size);
    printf("- Execution time: %.3f milliseconds\n", elapsedTime);
    printf("- Throughput: %.3f million elements/second\n",
           size / (elapsedTime / 1000.0f) / 1000000.0f);

    // Cleanup resources
    free(h_values);
    CHECK(cudaFree(d_values));
    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));

    return 0;
}


Writing task3.cu


In [27]:
!nvcc -arch=sm_70 -lcuda -lcudart task3.cu -o task3
!./task3 2048

Bitonic Sort - Array size: 2048 (must be power of two)

Initial array:
First 10 elements: 99 927 370 590 922 782 786 711 303 848 
Last 10 elements: 193 611 919 45 899 866 757 478 201 755 

Sorting verification: SUCCESS

Sorted array:
First 10 elements: 1 2 2 2 2 3 4 4 5 5 
Last 10 elements: 997 997 998 998 998 999 999 999 999 999 

Performance metrics:
- Array size: 2048 elements
- Execution time: 0.331 milliseconds
- Throughput: 6.195 million elements/second
